**Group members** :
- Luis Felipe Acosta Zavaleta
- Alejandro Mosquera Ospina
- Raúl Emerson Amao Guerra
- Manuel Antonio Gil Soto Pisco
- Ruth Marina Chávez Pacheco

In [1]:
#Libreries

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO
#from selenium.webdriver.chrome.service import Service
#from selenium.webdriver.chrome.options import Options

### My part_Raúl

In [10]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
tables_list = []

# We are selecting the last 5 elections
for k in range(21, 26):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    tables_list.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
final_table = pd.concat(tables_list, ignore_index=True)

# Save the final table to an Excel file
final_table.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 21:
             Elección ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1950   PARTIDO RESTAURADOR       550779


Data for election number 22:
             Elección        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1945  FRENTE DEMOCRATICO NACIONAL       305590
1  PRESIDENCIAL 1945         UNION REVOLUCIONARIA       150720


Data for election number 23:
             Elección   ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1939  CONCENTRACION NACIONAL       262971
1  PRESIDENCIAL 1939       FRENTE PATRIOTICO        76222


Data for election number 24:
             Elección     ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1936  PARTIDO SOCIAL DEMOCRATA        77042
1  PRESIDENCIAL 1936      UNION REVOLUCIONARIA        57838
2  PRESIDENCIAL 1936           FRENTE NACIONAL        46773
3  PRESIDENCIAL 1936       PARTIDO REPUBLICANO        30803


Data for election number 25:
             Elección       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PR

### Ruth: elections 1956-1985

In [8]:
driver = webdriver.Chrome()

driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")
url = 'https://infogob.jne.gob.pe/Eleccion'

elecciones_1985_1956 = []

# Select 5 years from 1956 to 1985
for x in range(16, 21):
    driver.get(url)
    time.sleep(3)
    proceso = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div").click()
    time.sleep(1)
    elecciones = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]").click()
    time.sleep(1)
    
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(x):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER) 

    eleccion = dropdown.text    
    verdatos = driver.find_element(By.XPATH,"/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button").click() # Select "Ver datos de elección"
    time.sleep(1)
    candidatosresultados = driver.find_element(By.XPATH,"/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]").click() # Select "Candidatos y resultados"
    time.sleep(1)

    tabla = driver.find_element(By.ID, 'CandidatosResultados').get_attribute('innerHTML')
    df = pd.read_html(StringIO(tabla))[0]
    df['Elección'] = eleccion
    
    df = df[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elecciones_1985_1956.append(df)
    time.sleep(2)            

    print(f"Data for election number {x}:\n", df)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
tabla_1985_1956 = pd.concat(elecciones_1985_1956, ignore_index=True)

# Save the final table to an Excel file
tabla_1985_1956.to_excel('Elecciones_1985_1956.xlsx', index=False)

Data for election number 16:
 Empty DataFrame
Columns: [Elección, ORGANIZACIÓN POLÍTICA, TOTAL VOTOS]
Index: []


Data for election number 17:
                                             Elección  \
0  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
1  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
2  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
3  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
4  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
5  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
6  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
7  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
8  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   
9  PRESIDENCIAL 1980\n[SELECCIONE]\nPRESIDENCIAL ...   

                           ORGANIZACIÓN POLÍTICA  TOTAL VOTOS  
0                                 ACCION POPULAR      1870864  
1                        PARTIDO APRISTA PERUANO      1129991  
2                      PARTIDO POPULAR CRISTIAN

### Part Luis Felipe - Elections 1990 (1st round), 1990 (2nd round), 1995, 2000 (1st round) & 2000 (2nd round)

In [11]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_1990_2000 = []

# We are selecting the last 5 elections
for k in range(11, 15):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_1990_2000.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
table_1990_2000 = pd.concat(elections_1990_2000, ignore_index=True)

# Save the final table to an Excel file
table_1990_2000.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 11:
                          Elección        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2000 - 2DA VUELTA  ALIANZA ELECTORAL PERU 2000      6041774
1  PRESIDENCIAL 2000 - 2DA VUELTA                 PERU POSIBLE      2086208
2  PRESIDENCIAL 2000 - 2DA VUELTA              VOTOS EN BLANCO       140773
3  PRESIDENCIAL 2000 - 2DA VUELTA                  VOTOS NULOS      3531730


Data for election number 12:
             Elección                       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2000                 ALIANZA ELECTORAL PERU 2000      5528394
1  PRESIDENCIAL 2000                                PERU POSIBLE      4460812
2  PRESIDENCIAL 2000         MOVIMIENTO INDEPENDIENTE SOMOS PERU       333049
3  PRESIDENCIAL 2000          AGRUPACION INDEPENDIENTE AVANCEMOS       246781
4  PRESIDENCIAL 2000                        SOLIDARIDAD NACIONAL       199813
5  PRESIDENCIAL 2000                     PARTIDO APRISTA PERUANO       152519
6  PRESIDENC

In [13]:
## Finally, we can see the required table
table_1990_2000

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2001,PERU POSIBLE,3871167
1,PRESIDENCIAL 2001,PARTIDO APRISTA PERUANO,2732857
2,PRESIDENCIAL 2001,ALIANZA ELECTORAL UNIDAD NACIONAL,2576653
3,PRESIDENCIAL 2001,FRENTE INDEPENDIENTE MORALIZADOR,1044207
4,PRESIDENCIAL 2001,ALIANZA ELECTORAL SOLUCION POPULAR,179243
5,PRESIDENCIAL 2001,PARTIDO RENACIMIENTO ANDINO,85436
6,PRESIDENCIAL 2001,PARTIDO PROYECTO PAIS,79077
7,PRESIDENCIAL 2001,TODOS POR LA VICTORIA,33080
8,PRESIDENCIAL 2001,VOTOS EN BLANCO,1260193
9,PRESIDENCIAL 2001,VOTOS NULOS,402436


### Alejandro Mosquera: Elections 2001-2011

In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_data = []

# Loop through years from 2001 to 2011
for year in range(6, 11):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)

    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Manually select the election for the current year (adjust the number of times to move down based on the year)
    for _ in range(year):  # Adjust the range based on the dropdown position
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.2)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(1)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    # Append the data to the list
    elections_data.append(table_data)

    time.sleep(2)

driver.quit()

# Concatenate all tables into a single DataFrame
elections_2001_2011 = pd.concat(elections_data, ignore_index=True)

# Display the final table
print(elections_2001_2011)

# Save the final table to an Excel file
elections_2001_2011.to_excel('Elecciones_Data.xlsx', index=False)

                          Elección               ORGANIZACIÓN POLÍTICA  \
0                PRESIDENCIAL 2011                           GANA PERU   
1                PRESIDENCIAL 2011                         FUERZA 2011   
2                PRESIDENCIAL 2011          ALIANZA POR EL GRAN CAMBIO   
3                PRESIDENCIAL 2011                        PERÚ POSIBLE   
4                PRESIDENCIAL 2011        ALIANZA SOLIDARIDAD NACIONAL   
5                PRESIDENCIAL 2011                 FONAVISTAS DEL PERU   
6                PRESIDENCIAL 2011                  DESPERTAR NACIONAL   
7                PRESIDENCIAL 2011           PARTIDO POLITICO ADELANTE   
8                PRESIDENCIAL 2011                     FUERZA NACIONAL   
9                PRESIDENCIAL 2011      JUSTICIA, TECNOLOGIA, ECOLOGIA   
10  PRESIDENCIAL 2006 - 2DA VUELTA             PARTIDO APRISTA PERUANO   
11  PRESIDENCIAL 2006 - 2DA VUELTA                   UNION POR EL PERU   
12  PRESIDENCIAL 2006 - 2DA VUELTA    

### Part Antonio Soto - Elections 2011-2021

In [13]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
base_url = 'https://infogob.jne.gob.pe/Eleccion'

# Load the existing Excel file
try:
    existing_data = pd.read_excel('Elecciones_Data.xlsx')
except FileNotFoundError:
    print("The file 'Elecciones_Data.xlsx' does not exist, a new one will be created.")
    existing_data = pd.DataFrame()  # Create an empty DataFrame if the file does not exist

# List to store new tables
elections_data = []

# List of elections to process
elections_list = [
    "PRESIDENCIAL 2021 - 2DA VUELTA",
    "PRESIDENCIAL 2021",
    "PRESIDENCIAL 2016 - 2DA VUELTA",
    "PRESIDENCIAL 2016",
    "PRESIDENCIAL 2011"
]

# Navigate and select the elections
for election in elections_list:
    driver.get(base_url)
    time.sleep(2)

    # Select "Tipo de Proceso"
    tipo_proceso_dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div')
    tipo_proceso_dropdown.click()
    time.sleep(2)

    # Use JavaScript to select the option if necessary
    driver.execute_script("arguments[0].click();", tipo_proceso_dropdown)
    ActionChains(driver).send_keys("ELECCIONES PRESIDENCIALES").send_keys(Keys.ENTER).perform()
    time.sleep(2)

    # Manually open the election dropdown
    election_dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    election_dropdown.click()
    time.sleep(1)

    # Find and click the specific election option
    option_xpath = f"//div[@class='item' and text()='{election}']"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, option_xpath))
    ).click()
    time.sleep(2)

    # Click "Ver Datos de la Elección"
    boton_ver_datos = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnVerDatos"]'))
    )
    boton_ver_datos.click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = election

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_data.append(table_data)

    time.sleep(2)

    print(f"Data for {election}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Combine the new tables into a single DataFrame
new_data = pd.concat(elections_data, ignore_index=True)

# Combine existing data with new data
data_2011_2021 = pd.concat([existing_data, new_data], ignore_index=True)

# Save the combined table to the Excel file
data_2011_2021.to_excel('Elecciones_Data.xlsx', index=False)

Data for PRESIDENCIAL 2021 - 2DA VUELTA:
                          Elección                 ORGANIZACIÓN POLÍTICA  \
0  PRESIDENCIAL 2021 - 2DA VUELTA  PARTIDO POLITICO NACIONAL PERU LIBRE   
1  PRESIDENCIAL 2021 - 2DA VUELTA                        FUERZA POPULAR   
2  PRESIDENCIAL 2021 - 2DA VUELTA                       VOTOS EN BLANCO   
3  PRESIDENCIAL 2021 - 2DA VUELTA                           VOTOS NULOS   

   TOTAL VOTOS  
0      8836380  
1      8792117  
2       121489  
3      1106816  


Data for PRESIDENCIAL 2021:
             Elección                        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2021         PARTIDO POLITICO NACIONAL PERU LIBRE      2724752
1  PRESIDENCIAL 2021                               FUERZA POPULAR      1930762
2  PRESIDENCIAL 2021                           RENOVACION POPULAR      1692279
3  PRESIDENCIAL 2021  AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL      1674201
4  PRESIDENCIAL 2021                               ACCION POPULAR      

In [15]:
## Finally, we can see the required table
Elecciones_final = pd.concat([final_table, tabla_1985_1956, table_1990_2000, elections_2001_2011, data_2011_2021], ignore_index=True)
Elecciones_final.to_excel('elecciones_final.xlsx', index=False)
Elecciones_final

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1950,PARTIDO RESTAURADOR,550779
1,PRESIDENCIAL 1945,FRENTE DEMOCRATICO NACIONAL,305590
2,PRESIDENCIAL 1945,UNION REVOLUCIONARIA,150720
3,PRESIDENCIAL 1939,CONCENTRACION NACIONAL,262971
4,PRESIDENCIAL 1939,FRENTE PATRIOTICO,76222
...,...,...,...
166,PRESIDENCIAL 2011,FONAVISTAS DEL PERU,37011
167,PRESIDENCIAL 2011,DESPERTAR NACIONAL,21574
168,PRESIDENCIAL 2011,PARTIDO POLITICO ADELANTE,17301
169,PRESIDENCIAL 2011,FUERZA NACIONAL,16831
